# MiniGo

## 使用 Swift 软件包管理器安装 MiniGo

In [1]:
%install-swiftpm-flags -c release -Xlinker -ltensorflow
%install '.package(url: "https://github.com/tensorflow/swift-models", .branch("tf-roadshow-19"))' MiniGo

Installing packages:
	.package(url: "https://github.com/tensorflow/swift-models", .branch("tf-roadshow-19"))
		MiniGo
With SwiftPM flags: ['-c', 'release', '-Xlinker', '-ltensorflow']
Working in: /tmp/tmp8wafobuy/swift-install
Fetching https://github.com/tensorflow/swift-models
Cloning https://github.com/tensorflow/swift-models
Resolving https://github.com/tensorflow/swift-models at tf-roadshow-19
[1/2] Compiling MiniGo Board.swift
/tmp/tmp8wafobuy/swift-install/package/.build/checkouts/swift-models/MiniGo/Models/GoModel.swift:56:21: warning: 'init(filterShape:strides:padding:dilations:activation:seed:)' is deprecated: This API will be removed after Swift for TensorFlow 0.4, please consider using
`init(filterShape:strides:padding:dilations:activation:filterIntializer:biasInitializer:)`
instead.
        self.conv = Conv2D(filterShape: filterShape, strides: strides, padding: padding)
                    ^
[2/3] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[3/3] Linki

## 导入现有模型

In [2]:
import TensorFlow
import MiniGo
import Foundation

In [3]:
system("""
    mkdir -p MiniGo
    gsutil cp 'gs://minigo-pub/v15-19x19/models/000939-heron.*' MiniGo/
    """)

Copying gs://minigo-pub/v15-19x19/models/000939-heron.data-00000-of-00001...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://minigo-pub/v15-19x19/models/000939-heron.index...                  
Copying gs://minigo-pub/v15-19x19/models/000939-heron.meta...                   
Copying gs://minigo-pub/v15-19x19/models/000939-heron.pb...                     
/ [4 files][266.2 MiB/266.2 MiB]    5.8 MiB/s                                   
Operation completed over 4 objects/266.2 MiB.                                    


0


In [4]:
let boardSize = 19
let modelConfig = ModelConfiguration(boardSize: boardSize)
var model = GoModel(configuration: modelConfig)

let reader = PythonCheckpointReader(path: "MiniGo/000939-heron")
model.load(from: reader)

## 设置游戏

In [8]:
let simulationCountForOneMove = 40

let config = GameConfiguration(
    size: boardSize,
    komi: 7.5,
    isVerboseDebuggingEnabled: false
)

let predictor = MCTSModelBasedPredictor(boardSize: boardSize, model: model)

let mctsConfiguration = MCTSConfiguration(
    gameConfiguration: config,
    simulationCountForOneMove: simulationCountForOneMove
)

## 玩一轮

In [9]:
try playOneGame(
    gameConfiguration: config,
    participants: [
        MCTSPolicy(participantName: "black", predictor: predictor,
                   configuration: mctsConfiguration),
        MCTSPolicy(participantName: "white", predictor: predictor,
                   configuration: mctsConfiguration),
    ])

: 